In [1]:
import os

In [2]:
from pathlib import Path
!pip install transformers[torch]
!pip install ftfy

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/opt/bwhpc/common/jupyter/tensorflow/2022-03-30/bin/python3.8 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/opt/bwhpc/common/jupyter/tensorflow/2022-03-30/bin/python3.8 -m pip install --upgrade pip' command.


In [3]:
DATA_PATH = Path("construction_images/data")

In [4]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
plt.rcParams["figure.figsize"] = (20,10)
import time

def imshow(img, bb=None):
    if bb is None:
        plt.imshow(img)
    else:
        x, y, a, b = bb
        fig, ax = plt.subplots()
        ax.imshow(img)
        rect = patches.Rectangle((x, y), a-x, b-y, linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
        plt.show()

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    return [lst[i:i + n] for i in range(0, len(lst), n)]

In [5]:
label_map = {
    "Deckenansicht": 0,
    "Detail": 1,
    "Revisionsoeffnung": 2,
    "Wandansicht": 3,
    "Kabel": 4,
    "Baugeraet": 5,
    "Trockenbau": 6,
    "Tuer": 7,
    "Fertigstellung": 8,
    "Brandschutz": 9,
    "Steckdose_Schalter": 10,
    "Beleuchtung": 11,
    "Lueftung": 12,
    "Fenster": 13,
    "Baumaterial": 14,
    "Aussenbereich": 15,
    "Trasse": 16,
    "Buero": 17,
    "Rohbau": 18,
    "Betonwand": 19,
    "Bauschutt": 20,
    "Abwasser": 21,
    "Heizungsrohr": 22,
    "Durchbruch": 23,
    "Erschliessung": 24,
    "Brandmelder": 25,
    "Keller": 26,
    "Mauerwerk": 27,
    "Sprinkler": 28,
    "Baunotiz": 29,
    "Installation": 30,
    "Bad": 31,
    "Deckenheizung": 32,
    "Technikzentrale": 33,
    "Parkhaus": 34,
    "Schacht": 35,
    "Bodenansicht": 36
 }
 
label_to_queries = {
    "Deckenansicht": "ceiling",
    "Detail": "detailed object",
    "Revisionsoeffnung": "inspection opening",
    "Wandansicht": "wall view",
    "Kabel": "cables",
    "Baugeraet": "construction equipment",
    "Trockenbau": "drywall",
    "Tuer": "door",
    "Fertigstellung": "completion",
    "Brandschutz": "fire protection",
    "Steckdose_Schalter": "socket",
    "Beleuchtung": "lighting",
    "Lueftung": "ventilation",
    "Fenster": "window",
    "Baumaterial": "building material",
    "Aussenbereich": "outdoor area",
    "Trasse": "route",
    "Buero": "office",
    "Rohbau": "shell construction",
    "Betonwand": "concrete wall",
    "Bauschutt": "construction waste",
    "Abwasser": "waste water",
    "Heizungsrohr": "heating pipe",
    "Durchbruch": "breakthrough",
    "Erschliessung": "accessibility",
    "Brandmelder": "fire detector",
    "Keller": "basement",
    "Mauerwerk": "masonry",
    "Sprinkler": "sprinkler",
    "Baunotiz": "building note",
    "Installation": "installation",
    "Bad": "bathroom",
    "Deckenheizung": "cealing heating",
    "Technikzentrale": "technical center",
    "Parkhaus": "parking lot",
    "Schacht": "shaft",
    "Bodenansicht": "bottom view"
 }
queries_to_label = {v: k for k, v in label_to_queries.items()}

In [ ]:
import requests
from PIL import Image
import torch
from tqdm import tqdm
import numpy as np
import json

from transformers import OwlViTProcessor, OwlViTForObjectDetection

processor = OwlViTProcessor.from_pretrained("google/owlvit-base-patch32")
model = OwlViTForObjectDetection.from_pretrained("google/owlvit-base-patch32")
texts = [list(label_to_queries.values())]
result_dict = dict()
images = os.listdir(DATA_PATH / "pretrain/images")
for j in tqdm(range(len(images))):
    img_path = images[j]
    image = Image.open(DATA_PATH / "pretrain/images" / img_path)
    inputs = processor(text=texts, images=image, return_tensors="pt")
    outputs = model(**inputs)

    # Target image sizes (height, width) to rescale box predictions [batch_size, 2]
    target_sizes = torch.Tensor([image.size[::-1]])
    # Convert outputs (bounding boxes and class logits) to COCO API
    results = processor.post_process(outputs=outputs, target_sizes=target_sizes)
    i = 0
    # Retrieve predictions for the first image for the corresponding text queries
    text = texts[i]
    boxes, scores, labels = results[i]["boxes"], results[i]["scores"], results[i]["labels"]

    # Print detected objects and rescaled box coordinates
    score_threshold = 0.1
    label_vec = np.zeros(len(label_map.keys()))
    for box, score, label in zip(boxes, scores, labels):
        box = [round(i, 2) for i in box.tolist()]
        if score >= score_threshold:
            label_vec[label_map[queries_to_label[text[label]]]] = 1
    result_dict[img_path] = list(label_vec)
    if j % 100 == 0:
        print("saving")
        with open(DATA_PATH / 'pretrain_labels.json', 'w') as f:
            json.dump(result_dict, f)

  0%|          | 1/33345 [00:05<50:42:52,  5.48s/it]

saving


  0%|          | 101/33345 [02:43<14:24:06,  1.56s/it]

saving


  1%|          | 201/33345 [05:19<14:14:38,  1.55s/it]

saving


  1%|          | 301/33345 [08:21<14:55:40,  1.63s/it]

saving


  1%|          | 397/33345 [10:53<14:19:12,  1.56s/it]